In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
books=pd.read_csv('BX-Books.csv',sep=";",encoding='latin-1',on_bad_lines='skip')
user=pd.read_csv('BX-Users.csv',sep=";",encoding='latin-1',on_bad_lines='skip')
rating=pd.read_csv('BX-Book-Ratings.csv',sep=";",encoding='latin-1',on_bad_lines='skip')

<ipython-input-4-cd928b32e53d>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('BX-Books.csv',sep=";",encoding='latin-1',on_bad_lines='skip')


In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
user.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
rating.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
books_renamed=books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Publisher':'publisher', "Image-URL-L": "img_url"})
#books_renamed.head()
books_renamed=books_renamed.drop(columns=['Image-URL-S','Image-URL-M'])
books_renamed.head()


,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
user_renamed=user.rename(columns={'User-ID':'user_id','Location':'location','Age':'age'})
user_renamed.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
rating_renamed=rating.rename(columns={'User-ID':'user_id','Book-Rating':'rating'})
rating_renamed.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
x=rating_renamed['user_id'].value_counts()>200
y=x[x].index
print(y.shape)
rating_renamed=rating_renamed[rating_renamed['user_id'].isin(y)]

(899,)


In [ ]:
rating_with_books=rating_renamed.merge(books_renamed,on='ISBN')
rating_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...


In [ ]:
number_rating=rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns={'rating':'number_of_ratings'},inplace=True)
number_rating.head()

,title,number_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [ ]:
final_rating=rating_with_books.merge(number_rating,on='title')
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,13


In [ ]:
final_ratings=final_rating[final_rating['number_of_ratings']>=50]
final_ratings.shape

(61853, 9)

In [ ]:
final_ratings=final_ratings.drop_duplicates(['user_id','title'])
final_ratings.shape

(59850, 9)

In [ ]:
books_pivot=final_ratings.pivot_table(columns='user_id',index='title',values='rating')
books_pivot.fillna(0,inplace=True)
books_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
_csr_matrix = csr_matrix(books_pivot)
_csr_matrix


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14961 stored elements and shape (742, 888)>

In [ ]:
model=NearestNeighbors(algorithm='brute')
model.fit(_csr_matrix)

NearestNeighbors(algorithm='brute')

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

def recommend_books_with_url(book_name, books_pivot, books_renamed):
    """Recommends books similar to the given book name and provides their URLs.

    Args:
        book_name (str): The name of the book to find recommendations for.
        books_pivot (pd.DataFrame): The pivot table of book ratings.
        books_renamed (pd.DataFrame): The DataFrame containing book information, including URLs.

    Returns:
        list: A list of recommended book titles and their URLs.
    """

    # Check if the book exists in the pivot table
    if book_name not in books_pivot.index:
        print(f"Book '{book_name}' not found in the dataset.")
        return []

    # Get the book ID
    book_id = np.where(books_pivot.index == book_name)[0][0]

    # Find nearest neighbors using KNN
    distances, suggestions = model.kneighbors(
        books_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=6
    )

    # Prepare recommendations with URLs
    recommendations = []
    for i in range(1, len(distances.flatten())):
        recommended_book_title = books_pivot.index[suggestions.flatten()[i]]
        # Get the URL for the recommended book
        img_url = books_renamed.loc[
            books_renamed["title"] == recommended_book_title, "img_url"
        ].iloc[0]
        recommendations.append((recommended_book_title, img_url))

    return recommendations


In [ ]:
recommend_books_with_url('Animal Farm',books_pivot,books_renamed)

[('Exclusive', 'http://images.amazon.com/images/P/0446604232.01.LZZZZZZZ.jpg'),
 ('Jacob Have I Loved',
  'http://images.amazon.com/images/P/0380564998.01.LZZZZZZZ.jpg'),
 ('Second Nature',
  'http://images.amazon.com/images/P/0399139087.01.LZZZZZZZ.jpg'),
 ('Pleading Guilty',
  'http://images.amazon.com/images/P/0446365505.01.LZZZZZZZ.jpg'),
 ('No Safe Place',
  'http://images.amazon.com/images/P/0345404777.01.LZZZZZZZ.jpg')]